In [1]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import KFold
import lightgbm as lgb

In [2]:
import os
os.getcwd()

'C:\\Files\\Jupyter\\IoT\\赛题二-初赛'

In [3]:
os.chdir('C:\\Files\\Jupyter\\IoT\\赛题二-初赛')

In [4]:
size_train=pd.read_csv('.\Train\Size.csv')
df_train_spc=pd.read_csv('.\Train\data_spc.csv')
df_test_spc=pd.read_csv('.\Test\data_spc.csv')

df_train_spc.drop(columns='remark', inplace=True)
df_test_spc.drop(columns='remark', inplace=True)

variables1=['Sensor1', 'Sensor2', 'Sensor3', 'IJ', 'Sensor5',
       'Sensor6', 'MouldTemp1', 'MouldTemp2', 'MouldTemp3', 'MouldTemp4',
       'MouldTemp5', 'MouldTemp9', 'MouldTemp10', 'MouldTemp11', 'MouldTemp12',
       'MouldTemp13', 'MouldTemp14', 'Sensor8', 'MouldFlow1', 'MouldFlow2',
       'MouldFlow3', 'SP']

In [6]:
TRAIN_ZIP=zipfile.ZipFile('.\Train\传感器高频数据.zip')
file_list=TRAIN_ZIP.namelist()
feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []

In [7]:
def feature_columns(variables):
    f_cols = []
    for v in variables:
        f_cols.append(v + '_mean')
    return f_cols

def stage_features(df, variables):
    avg = []
    tmp_df = df.loc[:, variables]
    for v in variables:
        tmp_avg = tmp_df[v].mean()
        avg.append(tmp_avg)    
    return np.array(avg)

In [8]:
feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []
for i,f in enumerate(file_list):
    df=pd.read_csv(TRAIN_ZIP.open(f))
    tmp = f.split('_')
    ti = tmp[2]
    mold_id = tmp[3].replace('.csv', '')
    times_.append(str(ti))
    mold_id_.append(int(mold_id))
    if len(df) == 0:
        features_[i] = [None for j in range(feature_n)]
    else:
        features_[i] = stage_features(df, variables1)
f_cols = feature_columns(variables1)

In [9]:
TRAIN_HIG = pd.DataFrame(features_, columns=f_cols)
TRAIN_HIG['Time'] = times_
TRAIN_HIG['Id'] = mold_id_
TRAIN_HIG = TRAIN_HIG[['Id', 'Time'] + f_cols]
TRAIN_ZIP.close()

In [11]:
TEST_ZIP=zipfile.ZipFile('.\Test\传感器高频数据.zip')
file_list=TEST_ZIP.namelist()
feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []
for i,f in enumerate(file_list):
    df=pd.read_csv(TEST_ZIP.open(f))
    tmp = f.split('_')
    ti = tmp[2]
    mold_id = tmp[3].replace('.csv', '')
    times_.append(str(ti))
    mold_id_.append(int(mold_id))
    if len(df) == 0:
        features_[i] = [None for j in range(feature_n)]
    else:
        features_[i] = stage_features(df, variables1)
f_cols = feature_columns(variables1)
TEST_HIG = pd.DataFrame(features_, columns=f_cols)
TEST_HIG['Time'] = times_
TEST_HIG['Id'] = mold_id_
TEST_HIG = TEST_HIG[['Id', 'Time'] + f_cols]
TEST_ZIP.close()

In [12]:
TRAIN_HIG.rename(columns={'Time':'spcTime'}, inplace=True)
TEST_HIG.rename(columns={'Time':'spcTime'}, inplace=True)

df_train_spc['spcTime']=df_train_spc['spcTime'].apply(int)
df_train_spc['spcTime']=df_train_spc['spcTime'].apply(str)
df_test_spc['spcTime']=df_test_spc['spcTime'].apply(int)
df_test_spc['spcTime']=df_test_spc['spcTime'].apply(str)

In [13]:
df_TRAIN=TRAIN_HIG.merge(df_train_spc, how='outer', on=['Id', 'spcTime']) 
df_TEST=TEST_HIG.merge(df_test_spc, how='outer', on=['Id', 'spcTime']) 
X_col=[i for i in df_TRAIN.columns if not i in ['Id', 'spcTime']]

In [14]:
K = 5
seed = 1234
skf = KFold(n_splits=K, shuffle=True, random_state=seed)
lgb_params = {
                        'boosting_type': 'gbdt',
                        'objective': 'regression',
                        'num_leaves': 2**5,
                        'subsample': 0.9,
                        'learning_rate': 0.05,
                        'seed': 2017,
                        'nthread': -1
             }

def mode_(size_i):
    predictions = np.zeros(len(X_test))
    for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
        print("fold {}".format(i))
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        lgb_train = lgb.Dataset(X_tr,y_tr)
        lgb_val = lgb.Dataset(X_val,y_val)
        num_round = 2000
        clf = lgb.train(lgb_params, lgb_train, num_round, valid_sets = [lgb_train, lgb_val],verbose_eval=50, 
                        early_stopping_rounds = 50)
        print('best iteration = ',clf.best_iteration)
        print("*"*100)
        predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / skf.n_splits
    return predictions

In [16]:
sub=pd.read_csv('sub_file.csv')
for i in ['size1','size2','size3']:
    print(i)
    X_train=df_TRAIN[X_col]
    y_train=size_train[i]
    X_test=df_TEST[X_col]
    sub[i]=mode_(i)

size1
fold 0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004944 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12657
[LightGBM] [Info] Number of data points in the train set: 13280, number of used features: 65
[LightGBM] [Info] Start training from score 300.065686
Training until validation scores don't improve for 50 rounds
[50]	training's l2: 0.000392653	valid_1's l2: 0.000450421
[100]	training's l2: 0.00032749	valid_1's l2: 0.000426317
[150]	training's l2: 0.000294051	valid_1's l2: 0.000423149
[200]	training's l2: 0.00026527	valid_1's l2: 0.000422581
Early stopping, best iteration is:
[187]	training's l2: 0.0002724	valid_1's l2: 0.000422187
best iteration =  187
****************************************************************************************************
fold 1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005839 seconds.
You can set `for

[300]	training's l2: 0.000217041	valid_1's l2: 0.0003864
[350]	training's l2: 0.000198527	valid_1's l2: 0.000386536
Early stopping, best iteration is:
[306]	training's l2: 0.000215026	valid_1's l2: 0.00038605
best iteration =  306
****************************************************************************************************
fold 4
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12658
[LightGBM] [Info] Number of data points in the train set: 13280, number of used features: 65
[LightGBM] [Info] Start training from score 200.005326
Training until validation scores don't improve for 50 rounds
[50]	training's l2: 0.000386223	valid_1's l2: 0.00041396
[100]	training's l2: 0.000315021	valid_1's l2: 0.000389904
[150]	training's l2: 0.000280283	valid_1's l2: 0.000385669
[200]	training's l2: 0.000254307	valid_1's l2: 0.000385183
[250]	training's l

In [25]:
sub

,Id,size1,size2,size3
0,56689,300.060775,199.993348,199.997687
1,56691,300.060208,199.965673,199.996706
2,56697,300.064945,199.964685,199.998738
3,56698,300.063678,199.964065,199.997576
4,56701,300.062362,199.960091,199.996479
...,...,...,...,...
3948,175911,300.040092,200.132680,200.045408
3949,175961,300.028621,200.127353,200.038473
3950,176111,300.025214,200.130948,200.046959
3951,176131,300.019501,200.126699,200.035191
